In [5]:
from pypdf import PdfReader

d:\CatchAI\catchAI\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [ ]:
def upload_pdf(pdf):
    reader = PdfReader(pdf)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def pdf_to_txt(text,pdf_name):
    new_text = text.replace("\n", " ").replace("  ", " ")
    with open(f"pdf/{pdf_name}.txt", "w", encoding="utf-8") as file:
        file.write(new_text)
    print(f"Texto guardado en pdf/{pdf_name}.txt")
    return new_text

def read_text(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    return text

    

In [7]:
reader = upload_pdf("pdf/CV_TomasFuentes.pdf")

print("Extracted Text:")
for line in reader.split('\n'):
    print(line)

pdf_to_txt(reader)

Extracted Text:
Tomás Fuentes Araya
INGENIERO CIVIL EN INFORMÁTICA Y TELECOMUNICACIONES
Universidad Diego Portales (UDP)
(+56) 9 8915 6457 | fuentesarayat@gmail.com |TomasFuentesA |tomasfa
Sobremi
IngenieroCivilenInformáticaconexperienciaenprocesamientodedatos,desarrollodepipelinesyparticipaciónenproyectosdeMachineLear-
ning.ApasionadoporlaCienciadeDatosysuaplicaciónprácticaparagenerarvalormedianteelanálisisymodeladodeinformación.Medestaco
pormicapacidadanalítica,aprendizajerápidoyhabilidadesinterpersonalesquefacilitaneltrabajoenequiposmultidisciplinarios.
Experiencia
JuniorDataScientist/DataAnalyst Santiago,Chile
SIMPLEDATA SPA Oct.2023-Abr.2025
•DesarrolloyoptimizaciónpipelinesdedatosutilizandoDatabricks(PySpark/SQL),mejorandotiemposdeprocesamientoyeficiencia.
•DesarrollodeETLenentornosUnixySQL,adaptandosolucionesalasnecesidadesdelcliente.
•DesarrollodemodelospredictivosenPythonutilizandoscikit-learn,desdelaexploracióndedatoshastalavalidacióncruzada.
Ayudantedeproyectos Santiago,

<_io.TextIOWrapper name='pdf/output.txt' mode='w' encoding='utf-8'>

In [8]:
def get_pages():
    reader = PdfReader("pdf/CV_TomasFuentes.pdf")
    return len(reader.pages)

In [9]:
get_pages()

1

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=750,
    chunk_overlap=750*0.15,  # 15% overlap
    length_function=len,
)

text = read_text("pdf/output.txt")

texts= text_splitter.create_documents([text])

texts = [doc.page_content for doc in texts]

texts

['Tomás Fuentes Araya\nINGENIERO CIVIL EN INFORMÁTICA Y TELECOMUNICACIONES\nUniversidad Diego Portales (UDP)\n\uf10b(+56) 9 8915 6457 | \uf0e0fuentesarayat@gmail.com |\uf092TomasFuentesA |\uf08ctomasfa\nSobremi\nIngenieroCivilenInformáticaconexperienciaenprocesamientodedatos,desarrollodepipelinesyparticipaciónenproyectosdeMachineLear-\nning.ApasionadoporlaCienciadeDatosysuaplicaciónprácticaparagenerarvalormedianteelanálisisymodeladodeinformación.Medestaco\npormicapacidadanalítica,aprendizajerápidoyhabilidadesinterpersonalesquefacilitaneltrabajoenequiposmultidisciplinarios.\nExperiencia\nJuniorDataScientist/DataAnalyst Santiago,Chile\nSIMPLEDATA SPA Oct.2023-Abr.2025',
 'Experiencia\nJuniorDataScientist/DataAnalyst Santiago,Chile\nSIMPLEDATA SPA Oct.2023-Abr.2025\n•DesarrolloyoptimizaciónpipelinesdedatosutilizandoDatabricks(PySpark/SQL),mejorandotiemposdeprocesamientoyeficiencia.\n•DesarrollodeETLenentornosUnixySQL,adaptandosolucionesalasnecesidadesdelcliente.\n•Desarrollodemodelospredi

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
vectors = embeddings.embed_documents(texts)
print(vectors)  # Cada elemento es el embedding de un texto

[[0.004519909620285034, -0.05165829136967659, -0.015276624821126461, 0.046510446816682816, -0.08729903399944305, -0.09175647050142288, 0.003511589951813221, 0.13476504385471344, -0.0090134022757411, -0.015068014152348042, 0.05749201774597168, 0.04398024454712868, -0.009685293771326542, 0.022286444902420044, -0.013941438868641853, -0.046365249902009964, -0.02221316285431385, -0.11708442121744156, 0.0032195691019296646, 0.024393681436777115, 0.0450398288667202, 0.018839599564671516, -0.014809336513280869, -0.01074312161654234, 0.0059072477743029594, -0.008274403400719166, -0.05192289873957634, -0.035046014934778214, 0.0007823792984709144, 0.009936450980603695, -0.0027468986809253693, 0.08517055958509445, 0.08213692903518677, -0.0011891741305589676, -0.05714240297675133, 0.02677534520626068, 0.004439202602952719, 0.005271723493933678, -0.10154729336500168, 0.034571465104818344, -0.02510560117661953, -0.0863550677895546, -0.007486189249902964, -0.09733086824417114, -0.011089440435171127, -

In [16]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_texts(texts, embeddings, persist_directory="./chroma_db")
vectorstore.persist()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
C:\Users\swamp\AppData\Local\Temp\ipykernel_12956\3361798056.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
